In [62]:
import pandas as pd
from scipy import stats
import numpy as np
import warnings
warnings.filterwarnings("ignore")
from tensorflow import keras
from tensorflow import keras
from scipy import stats
from tqdm import tqdm

In [117]:
!touch test.py

In [2]:
def encapsule(df):
    display(df.head(5))
    print( f'shape of df = {df.shape}' )
    return df

In [3]:
def tran(x):
    y = x[1:-1]
    return y.split(',')

def data_wrangle( df ):
    sample = df[['timestamp','raw_sensor_data']];
    sample['parameters'] = sample.raw_sensor_data.apply(tran);
    main = pd.DataFrame( sample.parameters.to_list() , columns = ['v1','v2','v3','v4','v5','v6','v7','v8','v9','v10','v11'] );
    main['timestamp'] = sample['timestamp'];
    main.drop( 'v1' , axis = 1 , inplace = True );
    main.set_index('timestamp' , inplace=True);
    return main;

In [81]:
def create_segments_and_labels(df, time_steps, step):

    # x, y, z acceleration as features
    N_FEATURES = 10
    # Number of steps to advance in each iteration (for me, it should always
    # be equal to the time_steps in order to have no overlap between segments)
    # step = time_steps
    segments = []
    labels = []
    for i in range(0, len(df) - time_steps, step):
        xs = df['v3'].values[i: i + time_steps]
        ys = df['v4'].values[i: i + time_steps]
        zs = df['v5'].values[i: i + time_steps]
        angle = df['v2'].values[i: i + time_steps]
        u5 = df['v6'].values[i: i + time_steps]
        u6= df['v7'].values[i: i + time_steps]
        u7= df['v8'].values[i: i + time_steps]
        u8= df['v9'].values[i: i + time_steps]
        u9= df['v10'].values[i: i + time_steps]
        u10= df['v11'].values[i: i + time_steps]
        # Retrieve the most often used label in this segment
        segments.append([xs, ys, zs,angle,u5,u6,u7,u8,u9,u10 ])

    # Bring the segments into a better shape
    reshaped_segments = np.asarray(segments, dtype= np.float32).reshape(-1, time_steps, N_FEATURES)
    reshaped = reshaped_segments.reshape(-1,350)
    return reshaped

In [5]:
  sample = encapsule(pd.read_csv('sord.csv'))


,id,user_id,email_id,raw_sensor_data,predicted_activity,actual_activity,timestamp,createdAt,updatedAt
0,27967,61f87f357d6217b5ed17047d,imove.mrt@gmail.com,"[4866.386144,14.911036744980214,2.48,0.28,-9.5...",sit,NaN,2022-02-02T16:13:39.218Z,2022-02-03T03:14:27.006Z,2022-02-03T03:14:27.006Z
1,27968,61f87f357d6217b5ed17047d,imove.mrt@gmail.com,"[4867.311019,14.911036744980214,2.47,0.27,-9.5...",sit,NaN,2022-02-02T16:13:40.143Z,2022-02-03T03:14:27.006Z,2022-02-03T03:14:27.006Z
2,27969,61f87f357d6217b5ed17047d,imove.mrt@gmail.com,"[4867.897186,14.911036744980214,2.5,0.24,-9.52...",sit,NaN,2022-02-02T16:13:40.729Z,2022-02-03T03:14:27.006Z,2022-02-03T03:14:27.006Z
3,27970,61f87f357d6217b5ed17047d,imove.mrt@gmail.com,"[4868.430045,14.911036744980214,2.48,0.26,-9.5...",sit,NaN,2022-02-02T16:13:41.262Z,2022-02-03T03:14:27.006Z,2022-02-03T03:14:27.006Z
4,27971,61f87f357d6217b5ed17047d,imove.mrt@gmail.com,"[4869.016053,14.911036744980214,2.46,0.26,-9.5...",sit,NaN,2022-02-02T16:13:41.848Z,2022-02-03T03:14:27.006Z,2022-02-03T03:14:27.006Z


shape of df = (5000, 9)


In [76]:
main = encapsule(data_wrangle(sample));


,v2,v3,v4,v5,v6,v7,v8,v9,v10,v11
timestamp,,,,,,,,,,
2022-02-02T16:13:39.218Z,14.911036744980214,2.48,0.28,-9.52,-0.0625,-0.0625,-0.125,21.0625,22.375,-40.6875
2022-02-02T16:13:40.143Z,14.911036744980214,2.47,0.27,-9.51,0.125,-0.125,0.125,20.6875,22,-39.375
2022-02-02T16:13:40.729Z,14.911036744980214,2.5,0.24,-9.52,-0.1875,-0.1875,0.125,20.375,21.5625,-40.1875
2022-02-02T16:13:41.262Z,14.911036744980214,2.48,0.26,-9.52,0,0.125,-0.125,21.0625,22.6875,-40.5625
2022-02-02T16:13:41.848Z,14.911036744980214,2.46,0.26,-9.55,-0.1875,-0.1875,-0.25,20.6875,22.375,-40.1875


shape of df = (5000, 10)


In [ ]:
def main():
  sample = pd.read_csv('sord.csv')
  main = data_wrangle(sample);
  reshaped = create_segments_and_labels(main,35,1)
  model = keras.models.load_model('/content/saved_model/my_model_2')
  currently_activity = ''
  number_of_windoews = len(sample) - 35
  from_ = sample.loc[0,'timestamp']
  Clsses = ['sit', 'std', 'wlk']
  final_list = []
  for each in tqdm(range(number_of_windoews)):
    y_hat =  model.predict(reshaped[each].reshape(-1,350))
    y_hat  = np.argmax(y_hat)
    y_hat = Clsses[y_hat]
    if((y_hat != currently_activity) & (y_hat != '') ):
      final_list.append( { 'start_time': from_ , 'end_time':sample.loc[each,'timestamp'] , 'activity': y_hat  } )
      currently_activity = y_hat
      from_ = sample.loc[each,'timestamp']




In [10]:
!tar -xvzf model_2.tar.gz

saved_model/my_model_2/
saved_model/my_model_2/saved_model.pb
saved_model/my_model_2/keras_metadata.pb
saved_model/my_model_2/assets/
saved_model/my_model_2/variables/
saved_model/my_model_2/variables/variables.index
saved_model/my_model_2/variables/variables.data-00000-of-00001


In [82]:
reshaped = create_segments_and_labels(main,35,1)

In [ ]:
model = keras.models.load_model('/content/saved_model/my_model_2')

In [101]:
currently_activity = ''
number_of_windoews = len(sample) - 35
from_ = sample.loc[0,'timestamp']
Clsses = ['sit', 'std', 'wlk']
final_list = []
for each in tqdm(range(number_of_windoews)):
  y_hat =  model.predict(reshaped[each].reshape(-1,350))
  y_hat  = np.argmax(y_hat)
  y_hat = Clsses[y_hat]
  if((y_hat != currently_activity) & (y_hat != '') ):
    final_list.append( { 'start_time': from_ , 'end_time':sample.loc[each,'timestamp'] , 'activity': y_hat  } )
    currently_activity = y_hat
    from_ = sample.loc[each,'timestamp']


100%|██████████| 4965/4965 [04:52<00:00, 16.97it/s]


In [115]:
repr(final_list)

"[{'start_time': '2022-02-02T16:13:39.218Z', 'end_time': '2022-02-02T16:13:39.218Z', 'activity': 'std'}, {'start_time': '2022-02-02T16:13:39.218Z', 'end_time': '2022-02-02T16:23:55.463Z', 'activity': 'wlk'}, {'start_time': '2022-02-02T16:23:55.463Z', 'end_time': '2022-02-02T16:23:56.437Z', 'activity': 'std'}, {'start_time': '2022-02-02T16:23:56.437Z', 'end_time': '2022-02-02T16:23:57.412Z', 'activity': 'wlk'}, {'start_time': '2022-02-02T16:23:57.412Z', 'end_time': '2022-02-02T16:23:57.900Z', 'activity': 'std'}, {'start_time': '2022-02-02T16:23:57.900Z', 'end_time': '2022-02-02T16:37:40.067Z', 'activity': 'wlk'}, {'start_time': '2022-02-02T16:37:40.067Z', 'end_time': '2022-02-02T16:37:41.042Z', 'activity': 'std'}, {'start_time': '2022-02-02T16:37:41.042Z', 'end_time': '2022-02-02T16:37:41.530Z', 'activity': 'wlk'}, {'start_time': '2022-02-02T16:37:41.530Z', 'end_time': '2022-02-02T16:37:42.991Z', 'activity': 'std'}, {'start_time': '2022-02-02T16:37:42.991Z', 'end_time': '2022-02-02T16:3

In [43]:
preds = []
for each in max_y_pred_test:
 preds.append(Clsses[each])


In [111]:
stats.mode(preds)

ModeResult(mode=array(['std'], dtype='<U3'), count=array([4726]))

In [104]:
sample.loc[np.where(np.roll(preds,1)!=preds)[0]]['timestamp']

1148    2022-02-02T16:23:55.463Z
1150    2022-02-02T16:23:56.437Z
1152    2022-02-02T16:23:57.412Z
1153    2022-02-02T16:23:57.900Z
2690    2022-02-02T16:37:40.067Z
2692    2022-02-02T16:37:41.042Z
2693    2022-02-02T16:37:41.530Z
2696    2022-02-02T16:37:42.991Z
2700    2022-02-02T16:37:44.942Z
2701    2022-02-02T16:37:45.430Z
2704    2022-02-02T16:37:46.892Z
2705    2022-02-02T16:37:47.379Z
2710    2022-02-02T16:37:49.865Z
2716    2022-02-02T16:37:52.839Z
2868    2022-02-02T16:39:07.330Z
2880    2022-02-02T16:39:13.180Z
2881    2022-02-02T16:39:13.667Z
2890    2022-02-02T16:39:18.055Z
2891    2022-02-02T16:39:18.542Z
2900    2022-02-02T16:39:22.977Z
2902    2022-02-02T16:39:23.953Z
2903    2022-02-02T16:39:24.441Z
2904    2022-02-02T16:39:24.976Z
2905    2022-02-02T16:39:25.464Z
3606    2022-02-02T16:52:44.530Z
3607    2022-02-02T16:52:45.018Z
3608    2022-02-02T16:52:45.503Z
3649    2022-02-02T16:53:15.821Z
3709    2022-02-02T16:55:40.368Z
3712    2022-02-02T16:55:41.829Z
3714    20

In [113]:
print('='*50 , 'From the Rule based' ,'='*50  )
print(repr(sample.predicted_activity.value_counts()))
print('='*50 , 'From the Model' ,'='*50  )
print(repr(Counter(preds)))

================================================== From the Rule based ==================================================
sit    4357
std     513
wlk     130
Name: predicted_activity, dtype: int64
================================================== From the Model ==================================================
Counter({'std': 4726, 'wlk': 239})


In [ ]:
Predict_a_block(block-of-data):

current_activity = nil



from = starttime (from the data)

timewindow array = [ from, to, activity]



for: how many sliding windows of 35 in the block-of-data -- len(b)

slice = window

prediction = model_m.predict(slice)

if (prediction != current_activity):

if (current_activity != nil):

add to the timewindow array (from , currentime, current_activity)

set current_activity = prediction

set from: currenttime



return the window array

[

    {from , to , sit},

{from, to, walk},

{....}

]